In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from google.colab import drive

In [174]:
%matplotlib inline
sns.set_style('darkgrid')
warnings.filterwarnings("ignore")

In [175]:
df = pd.read_csv("Train.csv")
df.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0


## Data Preprocessing

### Drop Columns

In [176]:
df.drop('user_id', axis=1, inplace=True) # Just a unique Identifier

In [177]:
df.drop('MRG', axis=1, inplace=True) # All data points have the same value

### Transforming Column "TENURE" into numerical values

In [178]:
df['TENURE'].value_counts()

,count
TENURE,
K > 24 month,2043201
I 18-21 month,45278
H 15-18 month,26006
G 12-15 month,14901
J 21-24 month,12725
F 9-12 month,9328
E 6-9 month,1839
D 3-6 month,770


In [179]:
mapping_dict = {'K > 24 month': 24, 'I 18-21 month': 18, 'H 15-18 month': 15, 'G 12-15 month': 12, 'J 21-24 month': 21, 'F 9-12 month': 9, 'E 6-9 month': 6, 'D 3-6 month': 3}
df['TENURE'] = df["TENURE"].apply(lambda x: mapping_dict[x])

In [180]:
df['TENURE'].value_counts()

,count
TENURE,
24,2043201
18,45278
15,26006
12,14901
21,12725
9,9328
6,1839
3,770


### New Column unlimited_pack

1: "TOP_PACK" Contains the substring "unlimited"

0: Otherwise

In [181]:
df['unlimited_pack'] = 0
df.loc[df['TOP_PACK'].str.contains("unlimited", case=False, na=False), 'unlimited_pack'] = 1

In [197]:
df[['unlimited_pack', "CHURN"]].corr()["CHURN"]

,CHURN
unlimited_pack,-0.161632
CHURN,1.000000


## New Column popular_pack

0: first quartile (x < 25)

1: second quartile (25 < x < 50)

2: third quartile (50 < x < 75)

3: fourth quartile (75 < x)

**Quartiles** of the value_counts of TOP_PACK column

We expect users of popular packages  

In [183]:
df['TOP_PACK'].value_counts().describe()

,count
count,140.000000
mean,8938.957143
std,33491.194518
min,1.000000
25%,4.500000
50%,112.000000
75%,1355.500000
max,317802.000000


In [184]:
low = df['TOP_PACK'].value_counts().describe()["25%"]
mid = df['TOP_PACK'].value_counts().describe()["50%"]
high = df['TOP_PACK'].value_counts().describe()["75%"]

val_counts = df['TOP_PACK'].value_counts()

def pack_pop(pack):
  if pack is np.nan:
    return -1

  val = val_counts[pack]

  if val < low:
    return 1
  elif val < mid:
    return 2
  elif val < high:
    return 3
  else:
    return 4

df["PACK_POP"] = df['TOP_PACK'].apply(pack_pop)

In [194]:
df[["PACK_POP", "CHURN"]].corr()["CHURN"]

,CHURN
PACK_POP,-0.444462
CHURN,1.000000


### NA>=10 Column

In [187]:
df[df['CHURN'] == 1].shape[0] / len(df)

0.1875473527052322

In [188]:
val_counts = df[df.isna().sum(axis=1) >= 10]['CHURN'].value_counts()
val_counts[1] / (val_counts[0] + val_counts[1])

np.float64(0.4525901246103)

In [189]:
df['NA>=10'] = False
df.loc[df.isna().sum(axis=1) >= 10, 'NA>=10'] = True

In [195]:
df[['NA>=10', 'CHURN']].corr()["CHURN"]

,CHURN
NA>=10,0.494754
CHURN,1.000000


## Null Value_counts

In [199]:
df["NULL_COUNTS"] = df.isna().sum(axis=1)

In [200]:
df[["NULL_COUNTS", "CHURN"]].corr()['CHURN']

,CHURN
NULL_COUNTS,0.530775
CHURN,1.000000


In [202]:
df[["NULL_COUNTS", "NA>=10"]].corr()['NULL_COUNTS']

,NULL_COUNTS
NULL_COUNTS,1.000000
NA>=10,0.957086


In [204]:
df.drop("NA>=10", axis=1, inplace=True)